In [1]:
import gym
import myosuite
from stable_baselines3 import PPO
from gym.envs.registration import register
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ENV Registration
ENV_NAME = 'Reach-v0'
MODEL_PATH = "C:/Personal/Second year/Sem 2/Project Course/myo_sim/finger/myofinger_v0.xml"
register(id=ENV_NAME,
        entry_point='myosuite.envs.myo.myobase.reach_v0:ReachEnvV0',
        max_episode_steps=200,
        kwargs={
            'model_path': MODEL_PATH,
            'target_reach_range': {'IFtip': ((0.2, 0.05, 0.20), (0.2, 0.05, 0.20)), },
            'normalize_act': True,
        }
)

MyoSuite:> Registering Myo Envs


c:\Users\hardi\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\envs\registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "c:\Users\hardi\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\envs\registration.py", line 594, in load_plugin_envs
    fn()
  File "c:\Users\hardi\AppData\Local\Programs\Python\Python310\lib\site-packages\shimmy\registration.py", line 303, in register_gymnasium_envs
    _register_dm_control_envs()
  File "c:\Users\hardi\AppData\Local\Programs\Python\Python310\lib\site-packages\shimmy\registration.py", line 63, in _register_dm_control_envs
    from shimmy.dm_control_compatibility import DmControlCompatibilityV0
  File "c:\Users\hardi\AppData\Local\Programs\Python\Python310\lib\site-packages\shimmy\dm_control_compatibility.py", line 20, in <module>
    from gymnasium.envs.mujoco.mujoco_rendering import MujocoRenderer
  File "c:\Users\hardi\A

In [2]:
env = gym.make('Reach-v0', seed=1)
obs = env.reset()

# Training Part
eval_env = gym.make('Reach-v0', seed=1)
eval_env.reset()

# Set up the callback
eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=500,
                             deterministic=True, render=False)

    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


c:\Users\hardi\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [3]:
model = PPO("MlpPolicy", env, verbose=1, device=device)
model.learn(total_timesteps=50000, callback=eval_callback)
model.save("myofinger_240404")

# Testing Trained RL Model
model.load("myofinger_240404")
done = False
while not done:
    act, _ = model.predict(obs)
    obs, reward, done, info = env.step(act)
    env.mj_render()
    print(done, act, obs)

Using cuda:0 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\hardi\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=500, episode_reward=-52.84 +/- 0.00
Episode length: 200.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 200      |
|    mean_reward     | -52.8    |
| time/              |          |
|    total_timesteps | 500      |
---------------------------------
New best mean reward!
Eval num_timesteps=1000, episode_reward=-52.84 +/- 0.00
Episode length: 200.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 200      |
|    mean_reward     | -52.8    |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------
Eval num_timesteps=1500, episode_reward=-52.84 +/- 0.00
Episode length: 200.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 200      |
|    mean_reward     | -52.8    |
| time/              |          |
|    total_timesteps | 1500     |
---------------------------------
E

KeyboardInterrupt: 